In [30]:
import pandas as pd
noro = pd.read_csv('/Users/evelyn/Desktop/Cubic Spline/noro_nooutliers.csv')

In [34]:
from itertools import combinations
a = []
features = ['TEMP','SLP','WDSP','PRCP','RH','SLP-STP','MXSPD-WDSP',
            'Median age','Immigrants','Households Median Income','%Food preparation','%Farming, fishing, and forestry','Population Density']
for p in combinations(features,9):
    a.append(p)
for p in combinations(features,10):
    a.append(p)
for p in combinations(features,11):
    a.append(p)
for p in combinations(features,12):
    a.append(p)

len(a)

1092

In [35]:
fea = pd.DataFrame()

In [40]:
import numpy as np
from progressbar import ProgressBar
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
import xgboost as xgb
import warnings
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

for i in tqdm(range(len(a))):
    fea.loc[i, "Features"] = str(a[i])
    X = noro[list(a[i])]
    y = noro['TotalCases']
   
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 25)
    warnings.filterwarnings("ignore")
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    y_pred_xg = model.predict(X_test)
    fea.loc[i, "xgboost"]=r2_score(y_test, y_pred_xg)
          

  0%|          | 0/1092 [00:00<?, ?it/s]

In [37]:
fea['xgboost'].max()

0.018624950537707297

In [43]:
index = fea[fea.xgboost == fea["xgboost"].max()].index.tolist()[0]
print(fea.loc[index,:])
print(fea.loc[index,"Features"])

Features    ('TEMP', 'WDSP', 'PRCP', 'RH', 'SLP-STP', 'MXS...
xgboost                                              0.018625
DT                                                  -0.263894
dt_rs                                                    40.0
Name: 1061, dtype: object
('TEMP', 'WDSP', 'PRCP', 'RH', 'SLP-STP', 'MXSPD-WDSP', 'Immigrants', 'Households Median Income', '%Food preparation', '%Farming, fishing, and forestry', 'Population Density')


In [41]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

for i in tqdm(range(len(a))):
    X = noro[list(a[i])]
    y = noro['TotalCases']
    dt1 = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 25)
    for rm in range(0,50):
        model2 = DecisionTreeRegressor(random_state = rm)
        model2.fit(X_train, y_train)
        y_pred = model2.predict(X_test)
        dt1.append(r2_score(y_test, y_pred))
    idx_rf = dt1.index(max(dt1))
    fea.loc[i, "DT"] = max(dt1)
    fea.loc[i, "dt_rs"] = idx_rf  

  0%|          | 0/1092 [00:00<?, ?it/s]

In [42]:
fea['DT'].max()

-0.008931569954027996

In [48]:
import numpy as np
from progressbar import ProgressBar
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

for i in tqdm(range(772,len(a))):
    X = noro[list(a[i])]
    y = noro['TotalCases']
    rf1 = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 25)
    for rm in range(0,50):
        model2 = RandomForestRegressor(random_state = rm)
        model2.fit(X_train, y_train)
        y_pred = model2.predict(X_test)
        rf1.append(r2_score(y_test, y_pred))
    idx_rf = rf1.index(max(rf1))
    fea.loc[i, "RF"] = max(rf1)
    fea.loc[i, "rf_rs"] = idx_rf   

  0%|          | 0/320 [00:00<?, ?it/s]

In [46]:
fea.to_csv('/Users/evelyn/Desktop/Cubic spline/noro912.csv')

In [49]:
fea['RF'].max()

-0.0025448120059490265

In [ ]:
# Compute RMSE
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import warnings
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

for i in tqdm(range(len(fea))):
    X = noro[list(a[i])]
    y = noro['TotalCases']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 25)
    
    #XGBOOST
    warnings.filterwarnings("ignore")
    model = xgb.XGBRegressor()
    model.fit(X_train, y_train)
    y_pred_xg = model.predict(X_test)
    mse = mean_squared_error(y_pred_xg, y_test)
    fea.loc[i, "rmse_xgboost"] = np.sqrt(mse)
    
    #DT
    model2 = DecisionTreeRegressor(random_state = int(fea.loc[i, "dt_rs"]))
    model2.fit(X_train, y_train)
    y_pred_dt = model2.predict(X_test)
    mse = mean_squared_error(y_pred_dt, y_test)
    fea.loc[i, "rmse_dt"] = np.sqrt(mse)
    
    #RF
    model2 = RandomForestRegressor(random_state = int(fea.loc[i, "rf_rs"]))
    model2.fit(X_train, y_train)
    y_pred_rf = model2.predict(X_test)
    mse = mean_squared_error(y_pred_rf, y_test)
    fea.loc[i, "rmse_rf"] = np.sqrt(mse)

In [54]:
index = fea[fea.xgboost == fea["xgboost"].max()].index.tolist()[0]
print(fea.loc[index,:])
print(fea.loc[index,"Features"])

Features        ('TEMP', 'WDSP', 'PRCP', 'RH', 'SLP-STP', 'MXS...
xgboost                                                  0.018625
DT                                                      -0.263894
dt_rs                                                        40.0
RF                                                      -0.033988
rf_rs                                                        23.0
rmse_xgboost                                            59.506283
rmse_dt                                                   67.5306
rmse_rf                                                 61.080578
Name: 1061, dtype: object
('TEMP', 'WDSP', 'PRCP', 'RH', 'SLP-STP', 'MXSPD-WDSP', 'Immigrants', 'Households Median Income', '%Food preparation', '%Farming, fishing, and forestry', 'Population Density')


In [55]:
index = fea[fea.DT == fea["DT"].max()].index.tolist()[0]
print(fea.loc[index,:])
print(fea.loc[index,"Features"])

Features        ('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'Immigra...
xgboost                                                 -0.091878
DT                                                      -0.008932
dt_rs                                                         9.0
RF                                                      -0.010348
rf_rs                                                         2.0
rmse_xgboost                                            62.767154
rmse_dt                                                 60.335953
rmse_rf                                                 60.378298
Name: 68, dtype: object
('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'Immigrants', '%Food preparation', '%Farming, fishing, and forestry', 'Population Density')


In [56]:
index = fea[fea.RF == fea["RF"].max()].index.tolist()[0]
print(fea.loc[index,:])
print(fea.loc[index,"Features"])

Features        ('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'Median ...
xgboost                                                 -0.087274
DT                                                       -0.22246
dt_rs                                                        23.0
RF                                                      -0.002545
rf_rs                                                         1.0
rmse_xgboost                                            62.634684
rmse_dt                                                 66.414458
rmse_rf                                                  60.14468
Name: 58, dtype: object
('TEMP', 'SLP', 'WDSP', 'PRCP', 'RH', 'Median age', 'Immigrants', '%Food preparation', '%Farming, fishing, and forestry')
